In [2]:
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [3]:
# read in the cleaned data for machine learning
ml_df = pd.read_csv('../CSV Files/Combined_ML.csv')
ml_df.head()

,month_date_yyyymm,postal_code,nielsen_hh_rank,hotness_rank,hotness_rank_mm,hotness_rank_yy,hotness_score,supply_score,demand_score,median_days_on_market_x,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,months_since_reference
0,2023-09-01,14450,998.0,1.0,-5.0,0.0,99.978698,99.974438,99.982958,7.0,...,439999.0,-0.1090,0.1193,26.0,0.1556,0.0000,0.1667,0.0691,-0.0476,73
1,2023-09-01,4106,3627.0,2.0,-2.0,0.0,99.923313,99.906271,99.940354,9.0,...,669841.0,0.0503,0.0367,27.0,0.2857,0.6875,0.2727,0.1675,0.0327,73
2,2023-09-01,14624,1837.0,3.0,-4.0,1.0,99.919052,99.965917,99.872188,8.0,...,293467.0,0.0970,0.1037,20.0,-0.0476,-0.1837,0.1765,0.0376,0.0602,73
3,2023-09-01,6111,3095.0,4.0,-1064.0,-15.0,99.897751,99.838105,99.957396,10.0,...,307199.0,-0.0268,-0.0417,65.0,-0.0299,-0.3085,3.2667,-0.0146,0.9274,73
4,2023-09-01,1970,884.0,5.0,-7.0,-5.0,99.893490,99.889230,99.897751,9.5,...,543278.0,0.0739,0.0137,35.0,0.4583,-0.3069,0.4000,0.0176,0.0667,73


In [4]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 926158 entries, 0 to 926157
Data columns (total 45 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   month_date_yyyymm                        926158 non-null  object 
 1   postal_code                              926158 non-null  int64  
 2   nielsen_hh_rank                          926158 non-null  float64
 3   hotness_rank                             926158 non-null  float64
 4   hotness_rank_mm                          926158 non-null  float64
 5   hotness_rank_yy                          926158 non-null  float64
 6   hotness_score                            926158 non-null  float64
 7   supply_score                             926158 non-null  float64
 8   demand_score                             926158 non-null  float64
 9   median_days_on_market_x                  926158 non-null  float64
 10  median_days_on_market_mm_x      

In [5]:
ml_df = ml_df.drop('month_date_yyyymm', axis=1)

In [6]:
# shift the hotness score to create teh target variable for the next period
ml_df['hotness_score_next_period'] = ml_df['hotness_score'].shift(-1)

# drop the last row since it will have NaN for the target variable
ml_df = ml_df[:-1]

# check that the column has been created.
ml_df.head()

,postal_code,nielsen_hh_rank,hotness_rank,hotness_rank_mm,hotness_rank_yy,hotness_score,supply_score,demand_score,median_days_on_market_x,median_days_on_market_mm_x,...,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,months_since_reference,hotness_score_next_period
0,14450,998.0,1.0,-5.0,0.0,99.978698,99.974438,99.982958,7.0,-0.333333,...,-0.1090,0.1193,26.0,0.1556,0.0000,0.1667,0.0691,-0.0476,73,99.923313
1,4106,3627.0,2.0,-2.0,0.0,99.923313,99.906271,99.940354,9.0,-0.100000,...,0.0503,0.0367,27.0,0.2857,0.6875,0.2727,0.1675,0.0327,73,99.919052
2,14624,1837.0,3.0,-4.0,1.0,99.919052,99.965917,99.872188,8.0,0.032258,...,0.0970,0.1037,20.0,-0.0476,-0.1837,0.1765,0.0376,0.0602,73,99.897751
3,6111,3095.0,4.0,-1064.0,-15.0,99.897751,99.838105,99.957396,10.0,-0.729730,...,-0.0268,-0.0417,65.0,-0.0299,-0.3085,3.2667,-0.0146,0.9274,73,99.893490
4,1970,884.0,5.0,-7.0,-5.0,99.893490,99.889230,99.897751,9.5,-0.173913,...,0.0739,0.0137,35.0,0.4583,-0.3069,0.4000,0.0176,0.0667,73,99.889230


In [7]:
# use binary encoder for the postal codes
binary_encoder = BinaryEncoder(cols=['postal_code'])

encoded_data = binary_encoder.fit_transform(ml_df['postal_code'])

ml_df_encoded = pd.concat([ml_df.drop('postal_code', axis=1), encoded_data], axis=1)

ml_df_encoded.head()

,nielsen_hh_rank,hotness_rank,hotness_rank_mm,hotness_rank_yy,hotness_score,supply_score,demand_score,median_days_on_market_x,median_days_on_market_mm_x,median_dom_mm_day,...,postal_code_5,postal_code_6,postal_code_7,postal_code_8,postal_code_9,postal_code_10,postal_code_11,postal_code_12,postal_code_13,postal_code_14
0,998.0,1.0,-5.0,0.0,99.978698,99.974438,99.982958,7.0,-0.333333,-3.50,...,0,0,0,0,0,0,0,0,0,1
1,3627.0,2.0,-2.0,0.0,99.923313,99.906271,99.940354,9.0,-0.100000,-1.00,...,0,0,0,0,0,0,0,0,1,0
2,1837.0,3.0,-4.0,1.0,99.919052,99.965917,99.872188,8.0,0.032258,0.25,...,0,0,0,0,0,0,0,0,1,1
3,3095.0,4.0,-1064.0,-15.0,99.897751,99.838105,99.957396,10.0,-0.729730,-27.00,...,0,0,0,0,0,0,0,1,0,0
4,884.0,5.0,-7.0,-5.0,99.893490,99.889230,99.897751,9.5,-0.173913,-2.00,...,0,0,0,0,0,0,0,1,0,1


In [8]:
# scale the non-encoded and non-target columns
features_scale = [col for col in ml_df_encoded.columns if 'postal_code' not in col and col != 'hotness_score_next_period']

scaler = StandardScaler()

ml_df_encoded[features_scale] = scaler.fit_transform(ml_df_encoded[features_scale])

print(ml_df_encoded.head())

   nielsen_hh_rank  hotness_rank  hotness_rank_mm  hotness_rank_yy  \
0        -1.207238     -1.623798        -0.013019         0.112626   
1        -0.716384     -1.623549        -0.011041         0.112626   
2        -1.050590     -1.623300        -0.012360         0.113058   
3        -0.815712     -1.623050        -0.711472         0.106151   
4        -1.228523     -1.622801        -0.014338         0.110468   

   hotness_score  supply_score  demand_score  median_days_on_market_x  \
0       2.054973      1.741926      1.730009                -1.677816   
1       2.052697      1.739557      1.728531                -1.616688   
2       2.052522      1.741630      1.726167                -1.647252   
3       2.051647      1.737189      1.729122                -1.586125   
4       2.051472      1.738965      1.727053                -1.601407   

   median_days_on_market_mm_x  median_dom_mm_day  ...  postal_code_5  \
0                   -1.313531          -0.219221  ...              0

In [9]:
# split the data in training and testing sets
# feature set
X = ml_df_encoded.drop('hotness_score_next_period', axis=1)
# target variable
y = ml_df_encoded['hotness_score_next_period']

# split training and testing by 80%/20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# check the resulting shape
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (740925, 58)
Test set shape: (185232, 58)


In [10]:
# initialize a linear regression model - first iteration

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

# The coefficients
print('Coefficients: \n', lin_reg.coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))


Coefficients: 
 [-5.07957621e-03 -6.08287164e-02  1.57927280e-02 -1.75147420e-02
 -1.76386061e+06  1.04317695e+06  1.04485011e+06  1.12626232e-01
 -5.41128850e-03 -1.01611502e-02  9.25879076e-03  1.34264018e-02
  4.77890373e-03  9.49637567e-02 -9.23885550e-01 -2.19201576e-03
  5.59861668e-03  5.73653770e-03  9.01271089e-03 -5.42841572e-03
 -4.76299407e-04 -2.63884934e-03 -1.24159927e-03  6.58289064e-05
  3.88319464e-03  4.60633390e-03 -6.75316589e-03 -6.03846018e-03
 -4.87230092e-04 -3.65637289e-03  1.42859771e-02 -3.55421319e-01
  1.01010359e+00 -1.61215084e-02 -4.62863423e-03  7.98674095e-03
  6.41787541e-04 -2.59453169e-04  2.27599039e-02  4.93539985e-03
 -4.01465577e-03  3.98961210e-03  7.93575810e-03  1.09191489e-02
  4.57509802e-03  8.96451747e-04  8.92506141e-04  6.74306879e-03
 -8.58519117e-04  7.87352717e-03  4.23284070e-03 -7.36980030e-03
  1.58950934e-03 -3.40428066e-03  1.59428432e-03  2.77610621e-03
 -2.08754596e-04 -3.16179094e-03]
Mean squared error: 0.80
Coefficient of 

In [11]:
# prepare 2nd iteration of the model
# remove columns that could cause overfitting

ml2_df = ml_df_encoded.drop(['hotness_rank_mm', 'hotness_rank_yy','median_days_on_market_mm_x',
       'median_dom_mm_day', 'median_days_on_market_yy_x', 'median_dom_yy_day','median_listing_price_mm_x',
       'median_listing_price_yy_x','active_listing_count_mm', 'active_listing_count_yy','new_listing_count_mm', 'new_listing_count_yy','price_increased_count_mm',
       'price_increased_count_yy','price_reduced_count_mm', 'price_reduced_count_yy','pending_listing_count_mm',
       'pending_listing_count_yy','median_listing_price_per_square_foot_mm',
       'median_listing_price_per_square_foot_yy','average_listing_price_mm', 'average_listing_price_yy','total_listing_count_mm',
       'total_listing_count_yy','pending_ratio_mm',
       'pending_ratio_yy', 'supply_score','pending_ratio',
       'demand_score',], axis=1)

ml2_df.head()

,nielsen_hh_rank,hotness_rank,hotness_score,median_days_on_market_x,median_listing_price_x,active_listing_count,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,...,postal_code_5,postal_code_6,postal_code_7,postal_code_8,postal_code_9,postal_code_10,postal_code_11,postal_code_12,postal_code_13,postal_code_14
0,-1.207238,-1.623798,2.054973,-1.677816,-0.094859,-0.536663,0.355741,-0.319757,-0.396439,-0.689621,...,0,0,0,0,0,0,0,0,0,1
1,-0.716384,-1.623549,2.052697,-1.616688,0.385962,-0.564729,0.217828,-0.319757,-0.565343,-0.641367,...,0,0,0,0,0,0,0,0,1,0
2,-1.050590,-1.623300,2.052522,-1.647252,-0.446018,-0.634892,0.355741,-0.319757,-0.565343,-0.713748,...,0,0,0,0,0,0,0,0,1,1
3,-0.815712,-1.623050,2.051647,-1.586125,-0.263602,-0.662957,0.217828,-0.319757,-0.734247,0.396094,...,0,0,0,0,0,0,0,1,0,0
4,-1.228523,-1.622801,2.051472,-1.601407,0.253679,-0.522631,0.907391,-0.319757,-0.565343,-0.544859,...,0,0,0,0,0,0,0,1,0,1


In [12]:
# split the data in training and testing sets
# feature set
X = ml2_df.drop('hotness_score_next_period', axis=1)
# target variable
y = ml2_df['hotness_score_next_period']

# split training and testing by 80%/20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# check the resulting shape
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (740925, 29)
Test set shape: (185232, 29)


In [13]:
# initialize a linear regression model - second iteration

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

# The coefficients
print('Coefficients: \n', lin_reg.coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [-5.13392678e-03 -5.96475703e-02  2.43156374e+01  8.07997255e-02
 -1.28657292e-03 -2.86792070e-02  1.80402791e-02  2.93378014e-04
  2.54197870e-03 -1.80637350e-02  1.71375315e-02 -1.46296347e-02
  2.80846201e-02  6.72554846e-03  4.21420222e-02  1.94458631e-02
  6.89328702e-03  6.29364766e-03  8.53478518e-03 -7.08193408e-04
  8.84362973e-03  4.69260929e-03 -6.52421847e-03  1.79469081e-03
 -3.98567276e-03  1.27062999e-03  2.51257315e-03  7.39265766e-04
 -3.45335764e-03]
Mean squared error: 0.80
Coefficient of determination: 1.00


In [ ]:
# Assuming 'df' is your DataFrame after preprocessing but before scaling
df_with_constant = add_constant(ml2_df)

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = df_with_constant.columns

# Calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_with_constant.values, i) for i in range(len(df_with_constant.columns))]

In [ ]:
# use ridge regression to regularize the model
ridge_model = Ridge(alpha=1.0)

ridge_model.fit(X_train, y_train)

ridge_scores = cross_val_score(ridge_model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
ridge_scores = np.sqrt(np.abs(ridge_scores))
ridge_score_mean = ridge_scores.mean()
ridge_score_std = ridge_scores.std()





In [ ]:
# The coefficients
print('Coefficients:', ridge_model.coef_)
# The mean squared error
print('Mean squared error:', mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination:', r2_score(y_test, y_pred))

In [ ]:
print("Cross-validation scores:", ridge_scores)
print("Mean of scores:", ridge_score_mean)
print("Standard deviation of scores:", ridge_score_std)

In [15]:
# use a random forest model instead of linear regression

# Split the data into a training set and a test set (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_regressor.predict(X_test)

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate the Coefficient of Determination (R^2)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Coefficient of Determination: {r2}")

Mean Squared Error: 0.39245578044551904
Coefficient of Determination: 0.9993383498315705


In [ ]:
# Get feature importances
importances = rf_regressor.feature_importances_

# Convert the importances into a DataFrame
feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': importances})

# Display the feature importances
print(feature_importance_df.sort_values('importance', ascending=False))

                                 feature    importance
2                          hotness_score  9.996233e-01
1                           hotness_rank  1.145849e-04
13                months_since_reference  4.039700e-05
3                median_days_on_market_x  3.729763e-05
5                   active_listing_count  3.457974e-05
11                 average_listing_price  2.518772e-05
0                        nielsen_hh_rank  2.460772e-05
10  median_listing_price_per_square_foot  1.629176e-05
12                   total_listing_count  1.541311e-05
4                 median_listing_price_x  1.116715e-05
17                         postal_code_3  8.412318e-06
9                  pending_listing_count  8.086053e-06
22                         postal_code_8  7.863711e-06
6                      new_listing_count  6.427801e-06
8                    price_reduced_count  4.694801e-06
19                         postal_code_5  4.393376e-06
28                        postal_code_14  3.002892e-06
26        

In [16]:
ml2_df.columns

Index(['nielsen_hh_rank', 'hotness_rank', 'hotness_score',
       'median_days_on_market_x', 'median_listing_price_x',
       'active_listing_count', 'new_listing_count', 'price_increased_count',
       'price_reduced_count', 'pending_listing_count',
       'median_listing_price_per_square_foot', 'average_listing_price',
       'total_listing_count', 'months_since_reference',
       'hotness_score_next_period', 'postal_code_0', 'postal_code_1',
       'postal_code_2', 'postal_code_3', 'postal_code_4', 'postal_code_5',
       'postal_code_6', 'postal_code_7', 'postal_code_8', 'postal_code_9',
       'postal_code_10', 'postal_code_11', 'postal_code_12', 'postal_code_13',
       'postal_code_14'],
      dtype='object')